In [1]:
import os
import json
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from utils.oct_dataset import OCTDataset
from utils.models import UnetNoPretraining, MedSAM, ResNetUNetWithAttention
from utils.lossfunctions import DiceLoss, DiceBCELoss
from utils.helper_methods import trial_dirname_creator
from utils.training_validating_testing import train_and_validate, test_best_model

In [ ]:
import numpy as np
import os
import tempfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from typing import Dict
import ray
from ray import train, tune
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray import tune, air
from ray.air import session
from ray.tune.search.optuna import OptunaSearch
from ray.train import RunConfig, CheckpointConfig
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import neptune
import segmentation_models_pytorch as smp
from segment_anything import sam_model_registry
os.environ["TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S"] = "0"

def train_model(config):
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")

    if config["model"] == "Unet":
        # Initialize model with the hyperparameters from the config
        net = smp.Unet(
            encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=1,                      # model output channels (number of classes in your dataset)
            activation="sigmoid",           # output activation (sigmoid for binary segmentation)
        )
        if config["freeze_encoder"]:
            for param in net.encoder.parameters():
                param.requires_grad = False

    elif config["model"] == "DeepLabV3+":
        net = smp.DeepLabV3Plus(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
            activation="sigmoid",
        )
        if config["freeze_encoder"]:
            for param in net.encoder.parameters():
                param.requires_grad = False 
    
    elif config["model"] == "MedSam":
        MedSAM_CKPT_PATH = config["origin_dir"]+"/medsam/medsam_vit_b.pth"
        sam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
        net = MedSAM(
                image_encoder=sam_model.image_encoder,
                mask_decoder=sam_model.mask_decoder,
                prompt_encoder=sam_model.prompt_encoder,
            ).to(device)
        
        if config["freeze_encoder"]:
            # Freeze the image encoder
            for param in net.image_encoder.parameters():
                param.requires_grad = False

    elif config["model"] == "AttentionUnet":
        net = ResNetUNetWithAttention()

        if(config["freeze_encoder"]):
            net = ResNetUNetWithAttention(freeze_entire_backbone=True)

    net.to(device)

    # Select optimizer based on the configuration
    if config["optimizer"] == "AdamW":
        optimizer = optim.AdamW(net.parameters(), lr=config["lr"])
    elif config["optimizer"] == "SGD":
        optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
    elif config["optimizer"] == "RMSprop":
        optimizer = optim.RMSprop(net.parameters(), lr=config["lr"])

    # Select loss function based on the configuration
    if config["loss_function"] == "DiceLoss":
        criterion = DiceLoss()
    elif config["loss_function"] == "DiceBCELoss":
        criterion = DiceBCELoss()
    elif config["loss_function"] == "BCELoss":
        criterion = nn.BCELoss()

    transform = transforms.Compose([
        transforms.Resize((256, 256), interpolation=Image.NEAREST),
        transforms.ToTensor(),
    ])

    # Load existing checkpoint through `get_checkpoint()` API.
    if train.get_checkpoint():
        loaded_checkpoint = train.get_checkpoint()
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    root_dir = config["root_dir"]
    folds= config["folds"]
    
    with open(os.path.join(root_dir, "metadata.csv"), "r") as f:
        metadata_df = pd.read_csv(f)
        skf = StratifiedKFold(n_splits=folds)
        splits = list(skf.split(metadata_df, metadata_df["unique_id"]))

    train_and_validate(root_dir, config, splits, 0, transform, optimizer, criterion, net, device)

In [ ]:
def main(num_samples, gpus_per_trial, epochs, smoke_test, folds):
    if smoke_test:
        root_dir = r"D:\OneDrive - Aarhus Universitet\9. Semester\Deep Learning\data_terumo_smoke_test"
        origin_dir = r"D:\OneDrive - Aarhus Universitet\9. Semester\Deep Learning"

    else:
        print("Using full dataset")
    
    config = {
        "root_dir": root_dir,
        "origin_dir": origin_dir,
        "lr": tune.loguniform(1e-6, 1e-2),
        "epochs": epochs,
        "smoke_test": smoke_test,
        "batch_size": tune.choice([4]),
        "optimizer": tune.grid_search(["AdamW", "SGD", "RMSprop"]),
        "folds": folds,
        "patience": 20,
        "loss_function": tune.grid_search(["DiceLoss", "BCELoss", "DiceBCELoss"]),
        "model": tune.grid_search(["AttentionUnet", "Unet", "DeepLabV3+", "MedSam"]),
        "freeze_encoder": tune.grid_search([True, False]),
    }

    # ASHA SCHEDULER, BUT WILL NOT BE USED
    # scheduler = ASHAScheduler(
    #     max_t=5,
    #     grace_period=5,
    #     reduction_factor=2
    # )

    # Define your checkpoint configuration
    checkpoint_config = CheckpointConfig(
        num_to_keep=1,  # Only keep the best checkpoint
        checkpoint_score_attribute="loss",  # The metric used to determine the best checkpoint
        checkpoint_score_order="min",  # Keep the checkpoint with the lowest loss
    )

    # Define the run config with the checkpoint config
    run_config = RunConfig(checkpoint_config=checkpoint_config)

    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_model),
            resources={"cpu": 2, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="dice_loss",
            mode="min",
            num_samples=num_samples,
            trial_dirname_creator=trial_dirname_creator,
        ),
        param_space=config,
        run_config=run_config,
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("dice_loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result, origin_dir)

main(num_samples=2, gpus_per_trial=1, epochs=2, smoke_test=True, folds=5)

(train_model pid=25192) [neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/OCTAA/OCTSegmenter/e/OCT-193
(train_model pid=25192) [1,    10] loss: 0.816
(train_model pid=25192) [1,    20] loss: 0.562


2024-12-01 16:44:32,206	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-12-01 16:44:32,262	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/johan/ray_results/train_model_2024-12-01_16-43-37' in 0.0550s.


(train_model pid=25192) [1,    30] loss: 0.770
(train_model pid=25192) Epoch [1/2], Training Loss: 0.7128


2024-12-01 16:44:42,601	INFO tune.py:1041 -- Total run time: 52.66 seconds (42.17 seconds for the tuning loop).
2024-12-01 16:44:42,602	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="C:/Users/johan/ray_results/train_model_2024-12-01_16-43-37", trainable=...)
2024-12-01 16:44:42,685	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 143 trial(s):
- train_model_0f465_00001: FileNotFoundError('Could not fetch metrics for train_model_0f465_00001: both result.json and progress.csv were not found at C:/Users/johan/ray_results/train_model_2024-12-01_16-43-37/trial_0f465_00001_lr=1.5e-05_opt=AdamW_bs=4_model=AttentionUnet_freeze=False_loss=DiceLoss')
- train_model_0f465_00002: FileNotFoundError('Could not fetch metrics for train_model_0f465_00002: both result.json and progress.csv were not found at C:/Users/johan/ray_results/train_model_2024-12-01_16-43-37/trial_0f465_00002_lr=1.9e-04_opt=

In [5]:
import os
from ray import tune
from ray.train import Result

storage_path = r"C:\Users\johan\ray_results"
exp_name = "train_model_cv_2024-11-26_18-30-35"
experiment_path = os.path.join(storage_path, exp_name)
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(experiment_path, trainable=train_model_cv)
result_grid = restored_tuner.get_results()

# Check if there have been errors
if result_grid.errors:
    print("One of the trials failed!")
else:
    print("No errors!")
    
# Get the result with the maximum test set `mean_accuracy`
best_result: Result = result_grid.get_best_result()

# Load the best checkpoint if it exists
if best_result.checkpoint:
    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")
else:
    raise ValueError("No checkpoint found for the best result.")

Loading results from C:\Users\johan\ray_results\train_model_cv_2024-11-26_18-30-35...
One of the trials failed!


OSError: [WinError 112] Failed copying 'C:/Users/johan/ray_results/train_model_cv_2024-11-26_18-30-35/trial_2b60a_00035_lr=1.0e-04_opt=RMSprop_bs=4/checkpoint_000004/checkpoint.pt' to 'C:/Users/johan/AppData/Local/Temp/checkpoint_tmp_eb1ac5460b9145e2847271a019179487/checkpoint.pt'. Detail: [Windows error 112] Der er ikke tilstrækkelig plads på disken.


In [ ]:
import os
from ray import tune
from ray.train import Result

storage_path = r"C:\Users\johan\ray_results"
exp_name = "train_model_cv_2024-11-26_16-31-22"
experiment_path = os.path.join(storage_path, exp_name)
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(experiment_path, trainable=train_model_cv)
result_grid = restored_tuner.get_results()

# Check if there have been errors
if result_grid.errors:
    print("One of the trials failed!")
else:
    print("No errors!")

num_results = len(result_grid)
print("Number of results:", num_results)

# Iterate over results
for i, result in enumerate(result_grid):
    if result.error:
        print(f"Trial #{i} had an error:", result.error)
        continue

    print(f"Trial #{i} finished successfully with a loss of: {result.metrics['loss']}")

results_df = result_grid.get_dataframe()
results_df[["training_iteration", "loss"]]

print("Shortest training time:", results_df["time_total_s"].min())
print("Longest training time:", results_df["time_total_s"].max())

ax = None
for result in result_grid:
    label = f"lr={result.config['lr']:.4f}, batch_size={result.config['batch_size']}, optimizer={result.config['optimizer']}, fold={result.config['fold']}"
    if ax is None:
        ax = result.metrics_dataframe.plot("training_iteration", "loss", label=label)
    else:
        result.metrics_dataframe.plot("training_iteration", "loss", ax=ax, label=label)
ax.set_title("Loss vs. Training Iteration for All Trials")
ax.set_ylabel("Loss")

# Get the result with the maximum test set `mean_accuracy`
best_result: Result = result_grid.get_best_result()

print("Best trial config: {}".format(best_result.config))

# Get the best trial's final loss and accuracy
best_loss = best_result.metrics["loss"]
print("Best trial final test set loss: {}".format(best_loss))
best_accuracy = best_result.metrics["accuracy"]
print("Best trial final test set accuracy: {}".format(best_accuracy))

# Load the best model
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

best_trained_model = UnetNoPretraining().to(device)

# Load the best checkpoint
checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

model_state, optimizer_state = torch.load(checkpoint_path, weights_only=True)
best_trained_model.load_state_dict(model_state)

# Set the model to evaluation mode
best_trained_model.eval()

# Load a sample image from the test dataset
root_dir = r"D:\OneDrive - Aarhus Universitet\9. Semester\Deep Learning\data_gentuity"
transform = transforms.Compose([
    transforms.Resize((256, 256), interpolation=Image.NEAREST),
    transforms.ToTensor(),
])
test_dataset = OCTDataset(root_dir, transform=transform, train=False, is_gentuity=True)
random_indices = np.random.choice(len(test_dataset), 1, replace=False)
sample_image, sample_mask, _, _ = test_dataset[random_indices[0]]  # Change the index to load a different sample

# Move the sample image to the appropriate device
sample_image = sample_image.to(device).unsqueeze(0)  # Add batch dimension

# Make a prediction
with torch.no_grad():
    prediction = best_trained_model(sample_image)

# Convert the prediction to a binary mask
predicted_mask = (prediction > 0.5).float()

# Plot the sample image, ground truth mask, and predicted mask
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

ax[0].imshow(sample_image.squeeze().permute(1, 2, 0).cpu().numpy())
ax[0].set_title("Sample Image")
ax[0].axis('off')

ax[1].imshow(sample_mask.squeeze().cpu().numpy(), cmap='gray')
ax[1].set_title("Ground Truth Mask")
ax[1].axis('off')

ax[2].imshow(predicted_mask.squeeze().cpu().numpy(), cmap='gray')
ax[2].set_title("Predicted Mask")
ax[2].axis('off')

plt.show()

2024-11-26 16:39:38,781	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 40 trial(s):
- train_model_cv_83259_00005: FileNotFoundError('Could not fetch metrics for train_model_cv_83259_00005: both result.json and progress.csv were not found at C:/Users/johan/ray_results/train_model_cv_2024-11-26_16-31-22/trial_83259_00005_lr=1.0e-04_opt=Adam_bs=4')
- train_model_cv_83259_00022: FileNotFoundError('Could not fetch metrics for train_model_cv_83259_00022: both result.json and progress.csv were not found at C:/Users/johan/ray_results/train_model_cv_2024-11-26_16-31-22/trial_83259_00022_lr=1.0e-04_opt=SGD_bs=4')
- train_model_cv_83259_00017: FileNotFoundError('Could not fetch metrics for train_model_cv_83259_00017: both result.json and progress.csv were not found at C:/Users/johan/ray_results/train_model_cv_2024-11-26_16-31-22/trial_83259_00017_lr=1.0e-04_opt=SGD_bs=4')
- train_model_cv_83259_00033: FileNotFoundError('Could not fetch metrics for train_model_cv_83259_00033: bo

Loading results from C:\Users\johan\ray_results\train_model_cv_2024-11-26_16-31-22...
One of the trials failed!
Number of results: 45


KeyError: 'loss'